In [1]:
import json
import os
from pprint import pprint
from makeNested import make_nestedJSON

In [2]:
# Make Nested JSON
with open('f2b_new_cats_vs_dogs_example.json') as f:
    inputs = json.load(f)
pprint(make_nestedJSON(inputs).parse())

{'dataset': {'path': '../data/dogs_and_cats', 'type': 'image'},
 'epochs': 5,
 'image': {'augment': {'height_shift_range': 0.2,
                       'horizontal_flip': 'True',
                       'rescale': 0.0039215,
                       'rotation_range': 40,
                       'width_shift_range': 0.2},
           'params': {'batch_size': 64,
                      'class_mode': 'binary',
                      'target_size': [200, 200]}},
 'layers': [{'activation': 'relu',
             'filters': 32,
             'input_shape': [200, 200, 3],
             'kernel_size': [3, 3],
             'name': 'Conv2D',
             'padding': 'same'},
            {'name': 'MaxPooling2D', 'pool_size': [2, 2]},
            {'name': 'Flatten'},
            {'activation': 'relu',
             'kernel_initializer': 'he_uniform',
             'name': 'Dense',
             'units': 128},
            {'activation': 'sigmoid', 'name': 'Dense', 'units': 1}],
 'loss': 'binary_crossentropy',
 'me

In [3]:
# Cell for all constants

TEST_DIR = ".." + os.sep + 'test' + os.sep
DATA_DIR = os.path.join(os.getcwd(), ".." + os.sep + 'data' + os.sep)
CATS_DIR = os.path.join(DATA_DIR, 'dogs_and_cats')

In [4]:


my_json = \
"""
{
	"layers": {
		"conv2d": {
			"filters": 32,
			"kernel_size": [3, 3],
			"activation": "relu",
			"padding": "same",
			"input_shape": [200, 200, 3]
		},

		"mp2": {
			"pool_size": [2, 2]
		},

        "flatten": {},

        "dense": {
            "units": 128,
            "activation": "relu",
            "kernel_initializer": "he_uniform"
        },

        "dense": {
            "units": 1,
            "activation": "sigmoid"
        }

	},

	"dataset": {
		"type": "image",
		"path": "../data/dogs_and_cats"
	},

	"image": {
		"augment": {
			"rotation_range": 40,
			"width_shift_range": 0.2,
			"height_shift_range": 0.2,
			"horizontal_flip": "True",
			"rescale": 0.0039215
		}
	},
	"params": {
		"target_size": [200, 200],
		"batch_size": 64,
		"class_mode": "binary"
	},

    "optimizer": "sgd",
    "loss": "binary_crossentropy",
    "metrics": [
        "accuracy"
    ],
    "epochs": 5,
    "verbose": 1,
    "plot": "True",
    "save_plots": "True"
}

"""

In [5]:
inputs = json.loads(my_json)

In [6]:
symbol_table = {
    'conv2d' : 'model.add(Conv2D())',
    'mp2' : 'model.add(MaxPooling2D())',
    'dense': 'model.add(Dense())',
    'flatten': 'model.add(Flatten())',
    'sgd': 'opt = SGD(lr=0.001, momentum=0.9)',
}

In [7]:
def get_imports():
    """
    imports the needed modules 
    """
    try:
        return \
        """
# File generated by DLMML Parser
import sys
import os
import numpy
import pandas
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
"""
    except Exception as e:
        print(e)

In [8]:
def init_sequential():
    """
    initializes the model
    """
    try:
        return \
'\nmodel = Sequential()\n'

    except Exception as e:
        print(e)

In [9]:
def image_input(inputs):
    """
    req in input json -
    dataset-path -> path to dataset
    image-augment -> params for image augmentation if required
    params -> params for building generator
    """
    base = inputs['dataset']['path']
    test_dir = os.path.join(base, 'test')
    train_dir = os.path.join(base, 'train')
    
    paths = \
'\nbase = \'{}\'\n\
train_dir = os.path.join(base, \'train\')\n\
test_dir = os.path.join(base, \'test\')\n'.format(inputs['dataset']['path'])

    i = \
'\naugment = {}\n\
kwargs = {}\n\n\
train_datagen = ImageDataGenerator(**augment)\n\
test_datagen = ImageDataGenerator(**augment)\n\
train_generator = train_datagen.flow_from_directory(train_dir, **kwargs)\n\
test_generator = test_datagen.flow_from_directory(test_dir, **kwargs)\n\n'.format(inputs['image']['augment'], inputs['params'])

    return paths+i


In [10]:
def parse(input_dict):
    """
    Parser which adds layers (mostly)
    """
    try:
        # init_sequential()
        input_dict = input_dict['layers']
        generated_code = ''

        #TODO: Take care of indentation and stuff if making functions
        #TODO: Ordering in dict (work around -> ordered_dict)

        for layer in input_dict.keys():
            curr_layer = symbol_table[layer]
            args = str(input_dict[layer])
            curr_layer = curr_layer[:-2] + '**' + args + curr_layer[-2:]
            generated_code += curr_layer + '\n'
        print()
        return generated_code
    except Exception as e:
        print(e)


In [11]:
def compile_model(inputs):
    """
    Compiles model along with adding optimizer
    """
    try:
        opt = inputs.get("optimizer", "sgd")
        opt_code = symbol_table[opt]
        # let loss and metrics be a compulsory fields for the user?
        loss = inputs.get("loss")
        metrics = str(inputs.get("metrics"))

        return '\n' + opt_code + '\n' + \
"model.compile(optimizer=opt, loss='{}', metrics={})".format(loss, metrics)

    except Exception as e:
        print(e)

In [12]:
def add_plots_and_summary(fname):
    return \
"""
def summarize_diagnostics(history, save_plots):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')

	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')

	# save plot to file
	if save_plots:
		filename = '{}'
		pyplot.savefig(filename + '_plot.png')
		pyplot.close()
""".format(fname)

In [13]:
def train_evaluate_model(inputs):
    """
    Adds code required to train and evaluate model
    Works only if:
        - train_data's generator is train_generator
        - test_data's generator is called test_generator
        - Only epochs and verbose taken from json
    """
    try:
            epochs = inputs.get('epochs', 20) #decide default
            verbose = inputs.get('verbose', 0)
            fit_generator = \
"""

print("\\n\\n ==========Fitting Model========== \\n")
history = model.fit_generator(
                    train_generator, 
                    steps_per_epoch=len(train_generator),
                    validation_data=test_generator, 
                    validation_steps=len(test_generator), 
                    epochs={}, 
                    verbose={}
                )
""".format(epochs, verbose)
            
            eval_generator = \
"""            

print("\\n\\n ==========Evalutating Model========== \\n")
_, acc = model.evaluate_generator(test_generator, steps=len(test_generator), verbose={})
print('\\n\\nACCURACY:  %.3f \\n\\n' % (acc * 100.0))
""".format(verbose)
            return fit_generator + eval_generator
    except Exception as e:
        print(e)


In [14]:
def plot_and_summarize_model(inputs):
    if eval(inputs.get('plot', False)):
        return \
"""

# Plotting graphs and Summarizing Model
summarize_diagnostics(history, {})
""".format(eval(inputs.get('save_plots', False)))

    else:
        return ''

In [15]:
def write_to_file(file_name, input_json):
    try:
        inputs = json.loads(input_json)
        with open(TEST_DIR + file_name, 'w') as f:
            f.write(get_imports())
            f.write(add_plots_and_summary("test"))
            f.write(image_input(inputs))
            f.write(init_sequential())
            f.write(parse(inputs))
            f.write(compile_model(inputs))
            f.write(train_evaluate_model(inputs))
            f.write(plot_and_summarize_model(inputs))
    except:
        print("Exception occured")

In [16]:
write_to_file('test.py', my_json)